# Commodity List
- **Reference**: https://site.financialmodelingprep.com/developer/docs#full-quote-list-commodities

## Step 1: Import Libraries

In [1]:
%run ../make_clean_names.py

In [2]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import logging
import concurrent.futures
from datetime import datetime
from typing import Dict, List
from concurrent.futures import ThreadPoolExecutor
import polars as pl

from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_session():
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    session.mount('https://', HTTPAdapter(max_retries=retries, pool_maxsize=10))
    return session

# Get API key from environment variables
FMP_API_KEY = os.getenv('FMP_API_KEY')
if not FMP_API_KEY:
    raise ValueError("FMP_API_KEY not found in environment variables")

## Step 2: Extract Data from FMP into Polars

In [3]:
start_date = '2020-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

def fetch_data(api_key: str, session: requests.Session) -> Dict:
    """Fetch historical forex data for a single pair with pair identifier"""
    url = f"https://financialmodelingprep.com/api/v3/quotes/commodity"
    params = {
        "apikey": api_key
    }
        
    try:
        response = session.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        return None

# Create a session
session = create_session()

# Execute fetching
data = fetch_data(FMP_API_KEY, session)

# Convert to Polars DataFrame if data exists
df = pl.DataFrame(data)

## Step 3: Clean Column Names

In [4]:
df = make_clean_names(df)

## Step 4: Write Polars to Parquet

In [5]:
# Define the output directory
output_dir = "../../../data/finance"

# Write the processed DataFrame to a Parquet file
df.write_parquet(f'{output_dir}/commodity_list.parquet')

## Step 5: Read Parquet (Validate)

In [6]:
# Validate the output by reading the Parquet file and displaying the first few rows
pl.scan_parquet(f'{output_dir}/commodity_list.parquet').head().collect()

shape: (0, 0)
┌┐
╞╡
└┘